# CV Enhancer

El objetivo es crear un agente que ayude a mejorar un currículum vitae (CV) agregando y enfocando las experiencias laborales y habilidades relevantes para un puesto de trabajo específico.

In [1]:
from google.adk.models.google_llm import Gemini
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


class CvSaverConfig:

    job_offer_summarizer_model: Gemini = Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    )

    writer_model: Gemini = Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    )


config = CvSaverConfig()

config.job_offer_summarizer_model


Gemini(model='gemini-2.5-flash-lite', speech_config=None, retry_options=HttpRetryOptions(
  attempts=5,
  exp_base=7.0,
  http_status_codes=[
    429,
    500,
    503,
    504,
  ],
  initial_delay=1.0
))

## Job Summarizer

Since the first input is a job offer, a model is needed to summarize the job offer in order to extract the key points. This will help to avoid spending too many tokens when delegating the task of improving the CV.

In [2]:
from pydantic import BaseModel, Field


class JobOfferSummarized(BaseModel):
    job_description: str = Field(..., description="Concise description of the job",)
    requirements: list[str] = Field(..., description="Key requirements listed clearly",)
    tech_stack: list[str] = Field(..., description="Technologies and tools mentioned",)


    __EXAMPLE__ = {
        "job_description": "Develop and maintain web applications.",
        "requirements": ["Python", "Django", "REST APIs"],
        "tech_stack": ["AWS", "Docker", "PostgreSQL"],
    }

assert JobOfferSummarized(  # noqa: S101
    **JobOfferSummarized.__EXAMPLE__
), "Example does not conform to schema"

job_offer_summ = JobOfferSummarized(**JobOfferSummarized.__EXAMPLE__)

job_offer_summ

JobOfferSummarized(job_description='Develop and maintain web applications.', requirements=['Python', 'Django', 'REST APIs'], tech_stack=['AWS', 'Docker', 'PostgreSQL'])

In [ ]:
import json

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.genai import types

PROMPT = f"""
You are a Job Offer Summarizer. Your only task is to read the provided job offer
and extract the key information such as the description, requirements, stack
and provide a concise summary. Do not add any additional information
(such as benefits, the title, the company...) or opinions.

The output must be only the summarized job offer without any additional
commentary.

Here is an example of the output format:
---
{json.dumps(JobOfferSummarized.__EXAMPLE__, indent=2)}
---
"""

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

job_offer_summarizer_agent = Agent(
    name="JobOfferSummarizerAgent",
    model=config.job_offer_summarizer_model,
    description=(
        "An agent that summarizes job offers by extracting key information and"
        " presenting it clearly to assist job seekers in understanding"
        " the opportunities."
    ),
    instruction=PROMPT,
    output_key="summarized_job_offer",
    output_schema=JobOfferSummarized,
)

# Expose the agent for external use
root_agent = job_offer_summarizer_agent

print("✅ Root Agent defined.")  # noqa: T201

In [5]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")  # noqa: T201

✅ Runner created.


In [6]:
response = await runner.run_debug(
    """
NeuralWorks es una compañía de alto crecimiento fundada hace aproximadamente 4 años. Estamos trabajando a toda máquina en cosas que darán que hablar.

Somos un equipo donde se unen la creatividad, curiosidad y la pasión por hacer las cosas bien. Nos arriesgamos a explorar fronteras donde otros no llegan: un modelo predictor basado en monte carlo, una red convolucional para detección de caras, un sensor de posición bluetooth, la recreación de un espacio acústico usando finite impulse response.

Estos son solo algunos de los desafíos, donde aprendemos, exploramos y nos complementamos como equipo para lograr cosas impensadas.

Trabajamos en proyectos propios y apoyamos a corporaciones en partnerships donde codo a codo combinamos conocimiento con creatividad, donde imaginamos, diseñamos y creamos productos digitales capaces de cautivar y crear impacto.

 Descripción del trabajo

El equipo de Analytics trabaja en diferentes proyectos que combinan volúmenes de datos enormes e IA, como detectar y predecir fallas antes que ocurran, optimizar pricing, personalizar la experiencia del cliente, optimizar uso de combustible, detectar caras y objetos usando visión por computador.

Como Data Scientist, trabajarás en conjunto con Machine Learning Engineers,Translators, Data Engineers entre otros perfiles, construyendo productos basados en datos que permitan llevar las prácticas de negocio al siguiente nivel.

El equipo de Analítica no opera como un equipo de Research separado, sino que está profundamente integrado con nuestros clientes y sus unidades de negocio, esto permite un feedback loop rápido para iterar, corregir y perfeccionar para finalmente conseguir el impacto de negocio.

En cualquier proyecto que trabajes, esperamos que tengas un gran espíritu de colaboración, una pasión por la innovación y el código y una mentalidad de automatización antes que procesos manuales.

Tu trabajo variará de proyecto a proyecto, pero siguiendo una estructura similar:

Identificar problemas de analítica que tengan un mayor impacto para la organización.
Familiarizarte con el problema junto a expertos, recabar información, opiniones y facts
Investigar las mejores prácticas de la industria.
Definir hipótesis de trabajo y approach técnico para resolver el problema, incluyendo el set de datos a utilizar, variables, periodo de tiempo, etc.
Recopilar datasets estructurados y no estructurados desde múltiples fuentes.
Realizar limpieza de datos y validar correctitud.
Aplicar modelos y algoritmos para minería de datos.
Analizar resultados e identificar patrones y tendencias.
Diseñar junto a Machine Learning Engineers soluciones que permitan capturar oportunidades.
Comunicar los resultados y discutirlos junto a expertos o tomadores de decisiones.
Algunos proyectos pueden incluir desafíos complejos en visión por computador.

 Calificaciones clave

Estudios de Ingeniería Civil en Computación o similar.
Experiencia previa en roles de Data Scientist, Data Engineer o similar.
Experiencia con Python.
Entendimiento de estructuras de datos con habilidades analíticas relacionadas con el trabajo con conjuntos de datos no estructurados, conocimiento avanzado de SQL, incluida la optimización de consultas.
Entendimiento de flujo de vida completo modelos de Machine Learning y productos de datos.
Habilidades analíticas, de diseño y resolución de problemas.
Habilidades de colaboración y comunicación.
Buen manejo comunicacional y skills colaborativos.
Buen manejo de inglés, sobre todo en lectura donde debes ser capaz de leer un paper, artículos o documentación de forma constante.

¡En NeuralWorks nos importa la diversidad! Creemos firmemente en la creación de un ambiente laboral inclusivo, diverso y equitativo. Reconocemos y celebramos la diversidad en todas sus formas y estamos comprometidos a ofrecer igualdad de oportunidades para todos los candidatos.

“Los hombres postulan a un cargo cuando cumplen el 60% de las calificaciones, pero las mujeres sólo si cumplen el 100%.” D. Gaucher , J. Friesen and A. C. Kay, Journal of Personality and Social Psychology, 2011.

Te invitamos a postular aunque no cumplas con todos los requisitos.

 Nice to have

Experiencia con servidores cloud (GCP, AWS o Azure), especialmente el conjunto de servicios de procesamiento de datos.
Experiencia usando pipelines de CI/CD y Docker.
Experiencia en visión por computador (Computer Vision), incluyendo uso de librerías como OpenCV, PyTorch/TensorFlow, YOLO, Detectron2 u otras.

 Beneficios

MacBook Air M2 o similar (con opción de compra hiper conveniente)
Bono por desempeño
Bono de almuerzo mensual y almuerzo de equipo los viernes
Seguro Complementario de salud y dental
Horario flexible
Flexibilidad entre oficina y home office
Medio día libre el día de tu cumpleaños
Financiamiento de certificaciones
Inscripción en Coursera con plan de entrenamiento a medida
Estacionamiento de bicicletas
Programa de referidos
Salida de “teambuilding” mensual
    """
)


 ### Created new session: debug_session_id

User > 
NeuralWorks es una compañía de alto crecimiento fundada hace aproximadamente 4 años. Estamos trabajando a toda máquina en cosas que darán que hablar.

Somos un equipo donde se unen la creatividad, curiosidad y la pasión por hacer las cosas bien. Nos arriesgamos a explorar fronteras donde otros no llegan: un modelo predictor basado en monte carlo, una red convolucional para detección de caras, un sensor de posición bluetooth, la recreación de un espacio acústico usando finite impulse response.

Estos son solo algunos de los desafíos, donde aprendemos, exploramos y nos complementamos como equipo para lograr cosas impensadas.

Trabajamos en proyectos propios y apoyamos a corporaciones en partnerships donde codo a codo combinamos conocimiento con creatividad, donde imaginamos, diseñamos y creamos productos digitales capaces de cautivar y crear impacto.

 Descripción del trabajo

El equipo de Analytics trabaja en diferentes proyectos que 

In [ ]:
def jaccard_simiarity(
    set1: set[str], set2: set[str], reference_set: set[str] | None = None
) -> float:
    if reference_set is None:
        reference_set = set1.union(set2)
    intersection = len(set1.intersection(set2))
    union = len(reference_set)
    if union == 0:
        return 0.0
    return intersection / union